## Summary

This notebook consists of several useful functions for live tracking the Twitch Content creator status, the types of games they play, activities etc.

Moreover, utilizing Twitch API's Helix package, we are capable of directly hunt for clips and content that the viewers of the streamers generate.
The API has a functionality of filtering clips within a specific timeframe, enabling us to target and hunt for clips that happen when very specific actions happen in the game

------------------------------------------------------

This notebook is only the 2nd in the process of creating an AI enabled API that monitors content creators playing game Valorant, recognizes significant events in the game, such as 4-5 kills by the protagonist, solo clutch etc hunts for the clips that have been captured during these events, downloads the clips and uploads to Youtube.

So the order of operations is:

1. AI - using voice recognition to identify significant game events
    - AI fixes the date and time of the event happening
2. Twitch API enables us to go and check the clips of the given content creator at the time of the event
    - We find and download the popular clip to the local machine/virtual server
3. We process and edit the video in python, adding an intro and outro and upload it automatically to Youtube
    - For every video we add customized captions, tags, descriptions etc

## Check streamer status

In [ ]:
import requests

## Given the registered API ID and secret code, we can send an HTML request and see what is the status of the given streamer
## We also generate an OAuth token to initialize the TwitchHelix
client_id = 'XYZ'
client_secret = 'YBB'
streamer_name = 'tarik'

body = {
    'client_id': client_id,
    'client_secret': client_secret,
    "grant_type": 'client_credentials'
}
r = requests.post('https://id.twitch.tv/oauth2/token', body)

#data output
keys = r.json();

print(keys)

headers = {
    'Client-ID': client_id,
    'Authorization': 'Bearer ' + keys['access_token']
}

print(headers)

stream = requests.get('https://api.twitch.tv/helix/streams?user_login=' + streamer_name, headers=headers)

stream_data = stream.json();

print(stream_data);

if len(stream_data['data']) == 1:
    print(streamer_name + ' is live: ' + stream_data['data'][0]['title'] + ' playing ' + stream_data['data'][0]['game_name']);
else:
    print(streamer_name + ' is not live');

## Twitch package

In [2]:
import twitch

In [3]:
client = twitch.TwitchHelix(client_id='TTT', oauth_token='AAA')

In [56]:
## Choosing specific time periods to retrieve the Twitch Clips
from datetime import datetime, timezone
from datetime import timedelta
local_time = datetime.now(timezone.utc).astimezone()
print()
print(local_time.isoformat())

## Set up the time delta
n = 1
# Add 15 minutes to datetime object
final_time = local_time + timedelta(days=-n)
final_time_str = final_time.strftime('%d/%m/%Y %H:%M:%S.%f')
print(final_time.isoformat())


2021-12-26T18:18:22.519008-08:00


In [40]:
## These two objects contain the top 100 popular clips (by No views) of the given creator
## returns a list of dictionaries, each dictionary contains such information as:
### game type, language, streamer name, views, link to the video etc

results = client.get_clips(broadcaster_id = '36340781', page_size = 100, started_at = final_time.isoformat(),
                          ended_at = local_time.isoformat())

games = client.get_clips(game_id = '516575', page_size = 100, started_at = final_time.isoformat(),
                          ended_at = local_time.isoformat())

In [55]:
## One of the examples
results[8]

{'id': 'ImpartialYawningAubergineFeelsBadMan-PupFQugQWb58rSD_',
 'url': 'https://clips.twitch.tv/ImpartialYawningAubergineFeelsBadMan-PupFQugQWb58rSD_',
 'embed_url': 'https://clips.twitch.tv/embed?clip=ImpartialYawningAubergineFeelsBadMan-PupFQugQWb58rSD_',
 'broadcaster_id': '36340781',
 'broadcaster_name': 'tarik',
 'creator_id': '43991108',
 'creator_name': 'ktodd76',
 'video_id': '1242726352',
 'game_id': '516575',
 'language': 'en',
 'title': 'When was the last time we were all together?',
 'view_count': 454,
 'created_at': datetime.datetime(2021, 12, 25, 3, 3, 43),
 'thumbnail_url': 'https://clips-media-assets2.twitch.tv/44168603308-offset-4848-preview-480x272.jpg',
 'duration': 29.3}

In [ ]:
### With this, we also can search for specific clips that a given user has created, in this example it's me
for i in results:
    if i['creator_name'] == "GuassianNormal":
        print(i['url'])
        break

https://clips.twitch.tv/CarelessSingleSpindleThunBeast-gi7ElfDKRJ-tRpjn


## Download the clip here

In [28]:
## Once we have the video URL, we can change up a little bit the structure and download it from the given browser directly
i['thumbnail_url'].split('-preview')[0] + ".mp4"

'https://clips-media-assets2.twitch.tv/AT-cm%7CYWZBd6vzvciVm9xXCkCNsQ.mp4'

In [36]:
## This code snippet takes the video URL and forces your browser to open it, directly downloading it into the default directory
import webbrowser

new = 2
url = i['thumbnail_url'].split('-preview')[0] + ".mp4"
webbrowser.open(url, new = new)

True